Importe de todas las librerías necesarias

In [ ]:
import random
import numpy as NumPy
import tensorflow as TensorFlow
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

Establecemos una semilla

In [ ]:
seed_value = 31
random.seed(seed_value)                             # Semilla para Python
NumPy.random.seed(seed_value)                       # Semilla para NumPy
TensorFlow.random.set_seed(seed_value)              # Semilla para TensorFlow/Keras

Función anexa que nos servirá para mostrar gráficas

In [47]:
def generatePlot(history):
    # Extracción de datos
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    # Creación del gráfico de accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training')
    plt.plot(epochs, val_acc, 'r', label='Validation')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Creación del gráfico de loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training')
    plt.plot(epochs, val_loss, 'r', label='Validation')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

### **Configuración de los parámetros iniciales**

Cada lote será de 16 fotos

Imagenes 150x150

Y se utilizará un 20% para el conjunto de validación

In [25]:
dataset_path = r".\Monkeys"
batch_size = 16
image_size = (150, 150)
rescale_factor = 1. / 255
validation_split_value = 0.2

### **Configuración de los conjuntos de Entrenamiento y Validación**

In [ ]:
data_generator = ImageDataGenerator(
    rescale=rescale_factor,
    validation_split=validation_split_value
)

train_generator = data_generator.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Visualización de las imagenes

In [ ]:
x_batch, y_batch = next(train_generator)
for i in range (0, 5):
    image = x_batch[i]
    plt.imshow(image)
    plt.show()

Configurar Early Stopping: A partir de 8 épocas sin mejora significativa de val_loss se detiene el entrenamiento.

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=8)

### **Configuración de los conjuntos de Entrenamiento y Validación**

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))
model.summary()

Compilación del modelo y Entrenamiento

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

epochs = 30

trainning = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[ early_stopping ]
)

Visualización de los valores de val_accuracy y val_loss con respecto a acuraccy y loss

In [ ]:
generatePlot(trainning)

score = model.evaluate(validation_generator, steps=50, verbose=0)  #'steps' es el número de lotes a evaluar

print('Test loss:', score[0])
print('Test accuracy:', score[1])